In [32]:
import pandas as pd
import plotly.graph_objects as go
import networkx as nx
import numpy as np

In [33]:
G = nx.random_geometric_graph(200, 0.125)


In [34]:
edge_x = []
edge_y = []
for edge in G.edges():
    x0, y0 = G.nodes[edge[0]]['pos']
    x1, y1 = G.nodes[edge[1]]['pos']
    edge_x.append(x0)
    edge_x.append(x1)
    edge_x.append(None)
    edge_y.append(y0)
    edge_y.append(y1)
    edge_y.append(None)

edge_x

[0.14070876825651923,
 0.2164485791098858,
 None,
 0.14070876825651923,
 0.21534129547334058,
 None,
 0.14070876825651923,
 0.23309311438615976,
 None,
 0.14070876825651923,
 0.018084295410322526,
 None,
 0.14070876825651923,
 0.14380633088072603,
 None,
 0.14070876825651923,
 0.12845749428927355,
 None,
 0.14070876825651923,
 0.07531833715297609,
 None,
 0.14070876825651923,
 0.2468317208329054,
 None,
 0.14070876825651923,
 0.18524971061447737,
 None,
 0.6912636919097422,
 0.6694287539594591,
 None,
 0.6912636919097422,
 0.6151235409629721,
 None,
 0.6912636919097422,
 0.6815527335324387,
 None,
 0.6912636919097422,
 0.7181625300453992,
 None,
 0.6912636919097422,
 0.6675794941190947,
 None,
 0.6912636919097422,
 0.6889018155636983,
 None,
 0.6912636919097422,
 0.7060991077162412,
 None,
 0.6912636919097422,
 0.6488080537904236,
 None,
 0.6912636919097422,
 0.6167900281319991,
 None,
 0.6912636919097422,
 0.65427947264377,
 None,
 0.6912636919097422,
 0.6778897367699734,
 None,
 0.69

In [35]:

edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines')

node_x = []
node_y = []
for node in G.nodes():
    x, y = G.nodes[node]['pos']
    node_x.append(x)
    node_y.append(y)

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        # colorscale options
        #'Greys' | 'YlGnBu' | 'Greens' | 'YlOrRd' | 'Bluered' | 'RdBu' |
        #'Reds' | 'Blues' | 'Picnic' | 'Rainbow' | 'Portland' | 'Jet' |
        #'Hot' | 'Blackbody' | 'Earth' | 'Electric' | 'Viridis' |
        colorscale='YlGnBu',
        reversescale=True,
        color=[],
        size=10,
        colorbar=dict(
            thickness=15,
            title=dict(
              text='Node Connections',
              side='right'
            ),
            xanchor='left',
        ),
        line_width=2))

In [36]:
edge_trace

Scatter({
    'hoverinfo': 'none',
    'line': {'color': '#888', 'width': 0.5},
    'mode': 'lines',
    'x': [0.14070876825651923, 0.2164485791098858, None, ..., 0.949414510596024,
          0.9345115167229232, None],
    'y': [0.06949729903388513, 0.16727074288215726, None, ...,
          0.23156996467159185, 0.2861370521530928, None]
})

In [37]:
node_adjacencies = []
node_text = []
for node, adjacencies in enumerate(G.adjacency()):
    node_adjacencies.append(len(adjacencies[1]))
    node_text.append('# of connections: '+str(len(adjacencies[1])))

node_trace.marker.color = node_adjacencies
node_trace.text = node_text

In [38]:
fig = go.Figure(data=[edge_trace],
             layout=go.Layout(
                title=dict(
                    text="<br>Network graph made with Python",
                    font=dict(
                        size=16
                    )
                ),
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                annotations=[ dict(
                    text="Python code: <a href='https://plotly.com/python/network-graphs/'> https://plotly.com/python/network-graphs/</a>",
                    showarrow=False,
                    xref="paper", yref="paper",
                    x=0.005, y=-0.002 ) ],
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                )
fig.show()

# Code

In [ ]:
# Load CSV file
csv_file = "networks_assignment.csv"
df = pd.read_csv(csv_file)


In [40]:
# Extract node labels
node_labels = df.columns[1:].tolist()  # Column headers as node names
source_nodes = df.iloc[:, 0].tolist()  # First column as source nodes

In [41]:
# Create a directed graph
G = nx.DiGraph()

# Add edges with weights
for i, src in enumerate(source_nodes):
    for j, dest in enumerate(node_labels):
        weight = df.iloc[i, j + 1]  # Get weight from adjacency matrix
        if weight > 0:  # Only add edges with nonzero weight
            G.add_edge(src, dest, weight=weight)

In [42]:
# Define node groups and colors
blue_nodes = {'N', 'D', 'F', 'I', 'S'}
green_nodes = {'BIH', 'GEO', 'ISR', 'MNE', 'SRB', 'CHE', 'TUR', 'UKR', 'GBR', 'AUS', 'HKG', 'USA'}
yellow_nodes = {'AUT', 'BEL', 'BGR', 'HRV', 'CZE', 'EST', 'FRA', 'DEU', 'GRC', 'HUN', 'IRL', 'ITA', 'LVA', 'LUX', 'NLD', 'PRT', 'ROU', 'SVK', 'SVN', 'ESP'}
node_colors = {}

for node in G.nodes():
    if node in blue_nodes:
        node_colors[node] = 'blue'
    elif node in green_nodes:
        node_colors[node] = 'green'
    elif node in yellow_nodes:
        node_colors[node] = 'yellow'
    else:
        node_colors[node] = 'red'  # Default color

In [43]:
# Define a pentagon layout for (D, F, I, N, S)
center_nodes = ["D", "F", "I", "N", "S"]
num_pentagon_nodes = len(center_nodes)
radius = 1.5  # Adjust for spacing

# Compute pentagon positions
pentagon_pos = {
    center_nodes[i]: (
        radius * np.cos(2 * np.pi * i / num_pentagon_nodes), 
        radius * np.sin(2 * np.pi * i / num_pentagon_nodes)
    ) 
    for i in range(num_pentagon_nodes)
}

In [44]:
# Compute positions for remaining nodes outside the pentagon
remaining_nodes = set(G.nodes()) - set(center_nodes)
pos = nx.spring_layout(G, seed=42)  # Compute positions for all nodes

# Adjust positions of external nodes to be farther from center
scale_factor = 2.5  # Moves external nodes outward
for node in remaining_nodes:
    pos[node] = (pos[node][0] * scale_factor, pos[node][1] * scale_factor)

# Override pentagon node positions
pos.update(pentagon_pos)

In [45]:
# Create Plotly traces
edge_x, edge_y, edge_weights = [], [], []
for edge in G.edges(data=True):
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_x.extend([x0, x1, None])  # None creates separate lines for each edge
    edge_y.extend([y0, y1, None])
    edge_weights.append(edge[2]["weight"])

# Create the edge trace
edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=1, color="black"),
    hoverinfo="text",
    text=[f"Weight: {w}" for w in edge_weights],
    mode="lines"
)

# Create the node trace
node_x, node_y, node_color, node_text = [], [], [], []
for node in G.nodes():
    x, y = pos[node]
    node_x.append(x)
    node_y.append(y)
    node_color.append(node_colors.get(node, 'red'))  # Default to red if missing
    node_text.append(node)

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode="markers+text",
    marker=dict(size=10, color=node_color),
    text=node_text,
    textposition="top center",
    hoverinfo="text"
)

In [46]:
# Create the final figure
fig = go.Figure(data=[edge_trace, node_trace])
fig.update_layout(
    title="Network Graph with Central Pentagon and Categorized Nodes",
    showlegend=False,
    hovermode="closest",
    margin=dict(b=10, l=10, r=10, t=40),
    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)
)

# Show the figure
fig.show()